In [1]:
import pandas as pd

df_train = pd.read_csv('./train_final_LLMScience.csv')

# Rename the column answer has 'actual_answer'
df_train.rename(columns={'answer': 'actual_answer'}, inplace=True)

In [2]:
df_train.head()

,id,prompt,A,B,C,D,E,actual_answer
0,0.0,Which of the following statements accurately d...,MOND is a theory that reduces the observed mis...,MOND is a theory that increases the discrepanc...,MOND is a theory that explains the missing bar...,MOND is a theory that reduces the discrepancy ...,MOND is a theory that eliminates the observed ...,D
1,1.0,Which of the following is an accurate definiti...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,A
2,2.0,Which of the following statements accurately d...,The triskeles symbol was reconstructed as a fe...,The triskeles symbol is a representation of th...,The triskeles symbol is a representation of a ...,The triskeles symbol represents three interloc...,The triskeles symbol is a representation of th...,A
3,3.0,What is the significance of regularization in ...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,C
4,4.0,Which of the following statements accurately d...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,D


In [11]:
df_train.loc[0]['E']

'MOND is a theory that eliminates the observed missing baryonic mass in galaxy clusters by imposing a new mathematical formulation of gravity that does not require the existence of dark matter.'

In [29]:
import cohere

co = cohere.Client('')

query = 'Which of the following statements accurately describes the impact of Modified Newtonian Dynamics (MOND) on the observed "missing baryonic mass" discrepancy in galaxy clusters?'
docs = [
    'MOND is a theory that reduces the observed missing baryonic mass in galaxy clusters by postulating the existence of a new form of matter called "fuzzy dark matter."',
    'MOND is a theory that increases the discrepancy between the observed missing baryonic mass in galaxy clusters and the measured velocity dispersions from a factor of around 10 to a factor of about 20.',
    'MOND is a theory that explains the missing baryonic mass in galaxy clusters that was previously considered dark matter by demonstrating that the mass is in the form of neutrinos and axions.',
    'MOND is a theory that reduces the discrepancy between the observed missing baryonic mass in galaxy clusters and the measured velocity dispersions from a factor of around 10 to a factor of about 2.',
    'MOND is a theory that eliminates the observed missing baryonic mass in galaxy clusters by imposing a new mathematical formulation of gravity that does not require the existence of dark matter.']
results = co.rerank(model="rerank-english-v2.0", query=query, documents=docs, top_n=5)


# Create a new list of dictionaries
parsed_list = []
for result in results:
    parsed_dict = {}
    # How to extract the text from the result object #
    text = result.document['text']
    score = result.relevance_score
    index = result.index
    parsed_dict['text'] = text
    parsed_dict['score'] = round(score,2)
    parsed_dict['index'] = index
    parsed_list.append(parsed_dict)

# Print the list of dictionaries
print(parsed_list)
    




[{'text': 'MOND is a theory that reduces the discrepancy between the observed missing baryonic mass in galaxy clusters and the measured velocity dispersions from a factor of around 10 to a factor of about 2.', 'score': 0.99, 'index': 3}, {'text': 'MOND is a theory that increases the discrepancy between the observed missing baryonic mass in galaxy clusters and the measured velocity dispersions from a factor of around 10 to a factor of about 20.', 'score': 0.99, 'index': 1}, {'text': 'MOND is a theory that eliminates the observed missing baryonic mass in galaxy clusters by imposing a new mathematical formulation of gravity that does not require the existence of dark matter.', 'score': 0.99, 'index': 4}, {'text': 'MOND is a theory that reduces the observed missing baryonic mass in galaxy clusters by postulating the existence of a new form of matter called "fuzzy dark matter."', 'score': 0.99, 'index': 0}, {'text': 'MOND is a theory that explains the missing baryonic mass in galaxy cluster

In [25]:
# Sample random 20 rows from the dataframe # 

df_sample = df_train.sample(20)
df_sample.head()

,id,prompt,A,B,C,D,E,actual_answer
4423,NaN,What is the primary focus of the Delaware Cont...,"Local, regional, as well as national and inter...",Regional artists only,Local artists only,Both local and regional artists,National and international artists only,A
2574,NaN,What is the distance in miles of the New Jerse...,1951 miles,117.20 miles,188.62 miles,22 months,It is not specified in the provided information.,E
2338,NaN,Which of the following media outlets have feat...,The New York Times and Variety magazine.,The Wall Street Journal and CNBC.,The Chicago Tribune and BBC News.,The Miami Herald and CBS News.,The Washington Post and ABC News.,A
5162,NaN,"Who directed the 1951 American Western film ""O...",Chill Wills,Charles Marquis Warren,Joseph Kane,Lorna Gray,Rod Cameron,C
5082,NaN,Question:,Prince Ignacy Massalski was a renowned physici...,Prince Ignacy Massalski served as a prominent ...,Prince Ignacy Massalski was a Polish-Lithuania...,"Prince Ignacy Massalski was a prolific author,...",Prince Ignacy Massalski was a renowned mathema...,C


In [31]:
map_index_option = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E'}
import time
# Loop through the above dataframe # 
for index, row in df_sample.iterrows():
    query = row['prompt']
    A = row['A']
    B = row['B']
    C = row['C']
    D = row['D']
    E = row['E']
    docs = [A, B, C, D, E]
    time.sleep(20)
    results = co.rerank(model="rerank-english-v2.0", query=query, documents=docs, top_n=5)
    predicted_answer = map_index_option[results[0].index]
    df_sample.loc[index, 'predicted_answer'] = predicted_answer

In [32]:
# Get the accuracy of the model based on predicted and actual answer # 
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score


actuals = df_sample['actual_answer'].tolist()
preds = df_sample['predicted_answer'].tolist()

accuracy = accuracy_score(actuals, preds)
print('Accuracy: %f' % accuracy)

precision = precision_score(actuals, preds, average='macro')
print('Precision: %f' % precision)

f1 = f1_score(actuals, preds, average='macro')
print('F1 score: %f' % f1)



Accuracy: 0.450000
Precision: 0.433333
F1 score: 0.430952
